In [ ]:
! pip install konlpy
import konlpy
from konlpy.tag import Kkma
from konlpy.utils import pprint
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 448 kB 38.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Mounted at /content/drive


In [ ]:
# 불용어 사전 데이터 불러오기
#########################################
# 자신이 맡은 섹션에 맞는 사전 가져오기!! #
# ex) IT&기술이면 stopword_IT&기술_okt.csv #
########################################

stopword = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/불용어모음/stopword_경제&사회_okt.csv', encoding = 'utf-8')

In [ ]:
# 불용어를 삭제할 데이터 불러오기(파일 이름이 token_kkma_okt로 끝나야겠죠?)
## 마찬가지로 데이터 확인 잘 하세요 ##

data = pd.read_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_토큰화_okt/경제&사회_20211022_token_kkma_okt.csv', encoding = 'utf-8')

In [ ]:
# 데이터프레임의 'token_okt' 칼럼이 하나의 스트링으로 연결된 형태라서 이를 떼어주는 작업입니다. 앞으로 데이터 쓸 때 매번 사용하게 될 예정

data['token_okt'] = data['token_okt'].apply(lambda k: eval(''.join(k)))

# 불용어 사전의 각 칼럼을 각각의 데이터프레임으로 분할
stop_words = stopword['stopword']
tags = stopword['tag']

In [ ]:
# 토큰화 된 본문이 (word, tag)의 꼴인 것을 떠올려봅시다.
# 필요한 단어만 남기려면 본문의 word는 불용어 사전에 있으면 안되고, tag는 우리가 원하는 tag만 남아있어야겠죠?
cleans = []
for i in range(len(data)):
    clean_words = [] 
    sentence = data['token_okt'][i]
    for word, pos in sentence:
      if word not in list(stop_words) and pos in ('Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Alpha', 'Unknown'):
        clean_words.append(word)
    cleans.append(clean_words)

# 위 작업을 거친 필요한 단어들을 새로운 칼럼에 'word'만 저장 ('word'만 저장하는 이유는 토픽모델링에서 그렇게 써야하기 때문에)
data['clean_words'] = cleans

In [ ]:
# 필요한 'word'만 남긴 칼럼의 각 row를 문장처럼 바꿔주는 작업입니다.
doclist = []
for i in range(len(data)):
  nouns = ' '.join(data['clean_words'][i])
  doclist.append(nouns)

# 역시 수행 결과를 새로운 칼럼에 저장
data['clean_as_sentence'] = doclist

########################################
# 여기까지가 불용어 제거 및 Tf-IDF 용으로 변환 #
######################################

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 1000)
x = tfidf_vectorizer.fit(data.clean_as_sentence)
tfidf_df = pd.DataFrame(x.transform(data.clean_as_sentence).toarray(), columns = sorted(tfidf_vectorizer.vocabulary_))

important_words = []
for i in range(0, len(tfidf_df)):
  a = []
  imp_word = tfidf_df.iloc[i, :].index[tfidf_df.iloc[i, :].values > 0.05]
  a.append(imp_word)
  important_words.append(a)

data['important_words'] = important_words

In [ ]:
# 이름 바꿔서 저장하세요!!

data.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/기사크롤링_TFIDF/경제&사회_20211022_TFIDF.csv', index = False, encoding = 'utf-8')